# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [1]:
import os
#Make this your results directory
os.chdir('..')
eval_path = 'multiclass/enhanced-definition/gemma2-27b/zero-shot.csv'

Open the Dataset

In [13]:
import pandas as pd
eval_path = 'multiclass/baseline/gemma2-27b/zero-shot.csv'
inference_df = pd.read_csv(eval_path)
inference_df

,Unnamed: 0,Prompt,Response
0,0,My husband works a lot which really helps our ...,Mental Filter \n
1,1,I used to get many strange looks for the thing...,No Distortion \n
2,2,Moved to another state left everything for my ...,Emotional Reasoning \n
3,3,"It has been more than a year now , I feel alon...",Personalization \n
4,4,"My sister has autism spectrum disorder, she al...",Mind Reading \n
...,...,...,...
501,501,From India: My brother is 40 years old and he ...,Overgeneralization \n
502,502,From the U.S.: I was sexually abused and raped...,No Distortion \n
503,503,My grandsons personality has changed in every ...,Emotional Reasoning \n
504,504,From Egypt: I was diagnosed with OCD by my doc...,Should Statements \n


Fix Common Variations

In [14]:
inference_df['Response'] = inference_df['Response'].apply(lambda x: x.lower())
inference_df['Response'] = inference_df['Response'].apply(lambda x: x.split('\n')[0])
inference_df['Response'] = inference_df['Response'].apply(lambda x: x.strip(".'\n "))
inference_df

,Unnamed: 0,Prompt,Response
0,0,My husband works a lot which really helps our ...,mental filter
1,1,I used to get many strange looks for the thing...,no distortion
2,2,Moved to another state left everything for my ...,emotional reasoning
3,3,"It has been more than a year now , I feel alon...",personalization
4,4,"My sister has autism spectrum disorder, she al...",mind reading
...,...,...,...
501,501,From India: My brother is 40 years old and he ...,overgeneralization
502,502,From the U.S.: I was sexually abused and raped...,no distortion
503,503,My grandsons personality has changed in every ...,emotional reasoning
504,504,From Egypt: I was diagnosed with OCD by my doc...,should statements


Create Mapping Function

In [15]:
def catch_responses(x):
    match x:
        case 'no distortion': 
            return 0
        case 'emotional reasoning': 
            return 1
        case 'overgeneralization': 
            return 2
        case 'mental filter':
            return 3
        case 'should statements':
            return 4
        case 'all-or-nothing thinking':
            return 5
        case 'mind reading': 
            return 6
        case 'fortune-telling':
            return 7
        case 'fortune telling':
            return 7
        case 'magnification': 
            return 8
        case 'personalization': 
            return 9
        case 'labeling': 
            return 10
        case _:
            print(x)
            return -1


Map common desired input, display any that are undesired for fine handling... For me, any denial should be thrown away (by being labelled -1).

In [16]:
inference_df['Response'] = inference_df['Response'].apply(catch_responses)

minimization
confirmation bias


Gather Gold Data

In [17]:
gold_dominant_data = pd.read_csv('../datasets/test.csv')['Dominant Distortion']\
    .apply(lambda x: x.lower())\
    .apply(catch_responses)
gold_secondary_data = pd.read_csv('../datasets/test.csv')['Secondary Distortion (Optional)']\
    .apply(lambda x: x if x is None else str(x).lower())\
    .apply(catch_responses)

inference_df['gold_dominant'] = gold_dominant_data
inference_df['gold_secondary'] = gold_secondary_data

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [18]:
def match_either(row):
    if row['Response'] == -1:
        row['gold'] = row['gold_dominant']
        return row
    if row['Response'] == row['gold_secondary']:
        row['gold'] = row['gold_secondary']
    else:
        row['gold'] = row['gold_dominant']
    return row

In [19]:
inference_df = inference_df.apply(match_either, axis=1)

Throw away non-responses

In [20]:
#inference_df = inference_df[inference_df['Response'] != -1]

Compute Accuracy and F1

In [21]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
f1_macro = f1_score(inference_df['gold'], inference_df['Response'], average=None)
for i, f1 in enumerate(f1_macro):
    print(f1)
print()
print(f1_score(inference_df['gold'], inference_df['Response'], labels=range(0,11), average='macro'))
print(accuracy_score(inference_df['gold'], inference_df['Response']))
#print(f"F1-Score (Weighted): {f1_score(inference_df['gold'], inference_df['Response'], average='weighted')}")

Evaluated Model: baseline Prompt: gemma2-27b
0.0
0.2857142857142857
0.11764705882352941
0.25
0.08163265306122448
0.11570247933884298
0.14705882352941177
0.3779527559055118
0.2857142857142857
0.1694915254237288
0.22950819672131148
0.25

0.21003836947564836
0.22924901185770752


In [22]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
print(f"Accuracy: {accuracy_score(inference_df['gold_dominant'], inference_df['Response'])}")
f1_macro = f1_score(inference_df['gold_dominant'], inference_df['Response'], labels=[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], average=None)
for i, f1 in enumerate(f1_macro):
    print(f"F1-Score (Macro, Class {i}): {f1}")
print(f"F1-Score (Macro): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='macro')}")
print(f"F1-Score (Weighted): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='weighted')}")

Evaluated Model: baseline Prompt: gemma2-27b
Accuracy: 0.19960474308300397
F1-Score (Macro, Class 0): 0.0
F1-Score (Macro, Class 1): 0.2857142857142857
F1-Score (Macro, Class 2): 0.08955223880597014
F1-Score (Macro, Class 3): 0.22641509433962265
F1-Score (Macro, Class 4): 0.041666666666666664
F1-Score (Macro, Class 5): 0.1
F1-Score (Macro, Class 6): 0.14084507042253522
F1-Score (Macro, Class 7): 0.3333333333333333
F1-Score (Macro, Class 8): 0.19718309859154928
F1-Score (Macro, Class 9): 0.06896551724137931
F1-Score (Macro, Class 10): 0.2
F1-Score (Macro, Class 11): 0.225
F1-Score (Macro): 0.1590562754262785
F1-Score (Weighted): 0.21563753713680056
